<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/NLP_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

이 노트북은 Made With ML의 교육 파일 https://github.com/madewithml/basics/blob/master/notebooks/14_Embeddings/14_TF_Embeddings.ipynb 을 연습한 노트북입니다.

This notebook is for practice of https://github.com/madewithml/basics/blob/master/notebooks/14_Embeddings/14_TF_Embeddings.ipynb in Made With ML

# Data

## Load data


In [0]:
import urllib

In [0]:
DATA_FILE = 'harrypotter.txt'

In [0]:
# github repository 에서 데이터를 읽어 옴
url = "https://raw.githubusercontent.com/madewithml/basics/master/data/harrypotter.txt"
response = urllib.request.urlopen(url)
html = response.read()
with open(DATA_FILE, 'wb') as fp:
  fp.write(html) # 파일로 생성

## Preprocess

In [0]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [0]:
import nltk; nltk.download('punkt')

from tensorflow.keras.preprocessing.text import text_to_word_sequence

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


nltk.punkt 는 자연어 토큰화 모듈.
토큰화에 대한 자세한 설명은 [여기](https://wikidocs.net/21698)를 참조.
요약하면 주어진 corpus를 사용하기 쉽게 정제하는 것.
`tf.keras.preprocessing.text.text_to_word_sequence` 는 문장을 단어의 연속으로 바꿔 줌. 


In [0]:
SEED = 1234

In [0]:
tf.random.set_seed(SEED)

In [0]:
FILTERS = "!\"'#$%&()*+,_./:;<=>?@[\\]^_'{|}~"
LOWER = True

In [0]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
with open(DATA_FILE, encoding='cp1252') as fp:
  book = fp.read()
sentences = tokenizer.tokenize(book)
print(f"{len(sentences)} sentences")

15640 sentences


In [0]:
print (sentences[12])
sentences = [text_to_word_sequence(text=sentence,filters=FILTERS,lower=LOWER, split=" ") for sentence in sentences]
print(sentences[12])

They turned right, into a wide driveway that led off the lane.
['they', 'turned', 'right', 'into', 'a', 'wide', 'driveway', 'that', 'led', 'off', 'the', 'lane']


`' '` 을 기점으로 단어를 잘라 리스트로 변환함</br>
`lower` 옵션으로 소문자화 설정.

# Word2Vec

In [0]:
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [0]:
EMBEDDING_DIM = 100 # embedding 벡터
WINDOW = 5
MIN_COUNT = 3 # 3회 이하로 나온 단어는 무시
SKIP_GRAM = 1 # 0 = CBOW
NEGATIVE_SAMPLING = 20

In [0]:
w2v = Word2Vec(sentences=sentences,size = EMBEDDING_DIM, window=WINDOW, min_count = MIN_COUNT, sg = SKIP_GRAM, negative=NEGATIVE_SAMPLING)


In [0]:
print(w2v)

Word2Vec(vocab=4943, size=100, alpha=0.025)


In [0]:
w2v.wv.get_vector("potter")

array([-0.04382599, -0.27061546,  0.1651173 ,  0.11518585,  0.08209284,
        0.29084224,  0.41149804,  0.15002549,  0.15954678, -0.15583111,
        0.21830478,  0.33232954,  0.09824262,  0.13917533, -0.1396302 ,
       -0.25955015, -0.16083549,  0.02623774,  0.17311276, -0.0435692 ,
        0.24526185,  0.2566217 , -0.29358414, -0.06039868, -0.3891427 ,
        0.08209246, -0.04794239, -0.0095009 ,  0.05358456,  0.47284657,
        0.25039336,  0.01511135,  0.36297756,  0.28031012, -0.36281002,
        0.5458123 ,  0.07694073,  0.02720371,  0.02810892, -0.38364294,
        0.21260844,  0.26286042, -0.13364011,  0.12455881, -0.15269653,
        0.00127807,  0.5376269 , -0.01063405,  0.14180158,  0.06000757,
       -0.22659217, -0.02594471, -0.08970959,  0.07681642, -0.33292103,
       -0.27242723, -0.12953316,  0.41518903,  0.08366511,  0.08306436,
       -0.3427147 , -0.0542339 , -0.14626692,  0.19276895,  0.10108715,
       -0.5280813 ,  0.21871641, -0.0438563 ,  0.6344262 , -0.04

In [0]:
w2v.wv.most_similar(positive="scar", topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('pain', 0.9584817886352539),
 ('forehead', 0.9346109628677368),
 ('burning', 0.9248462915420532),
 ('cold', 0.924383282661438),
 ('burned', 0.921999454498291)]

In [0]:
w2v.wv.save_word2vec_format('model.bin', binary=True)
w2v = KeyedVectors.load_word2vec_format('model.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Pretrained embeddings

In [0]:
from gensim.scripts.glove2word2vec import glove2word2vec
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from urllib.request import urlopen
from zipfile import ZipFile

In [14]:
np.random.seed(SEED)

NameError: ignored

In [0]:
EMBEDDING_DIM = 100

In [0]:
def plot_embeddings(words, embeddings, pca_results):
  for word in words:
    index = embeddings.index2word.index(word)
    plt.scatter(pca_results[index, 0], pca_results[index, 1])
    plt.annotate(word, xy=(pca_results[index, 0], pca_results[index, 1]))
  plt.show()

In [0]:
resp = urlopen('http://nlp.stanford.edu/data/glove.6B.zip')
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()

In [0]:
embeddings_file = f"glove.6B.{EMBEDDING_DIM}d.txt"
zipfile.extract(embeddings_file)

In [0]:
with open(embeddings_file, 'r') as fp:
  for i in range(1,80):
    line = next(fp)
    values = line.split()
    word = values[0]
    embedding = np.asarray(values[1:], dtype='float32')
    print(f"word : {word}")
    # print(f"embedding: \n{embedding}")
    # print(f"embedding dim: {len(embedding)}")
    

In [0]:
# local 디렉토리에 저장
word2vec_output_file = f"{embeddings_file}.word2vec"
glove2word2vec(embeddings_file, word2vec_output_file)

In [0]:
# 임베딩 파일 오픈
glove = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [0]:
# (king - man) + woman = ? == man : king = woman : ?
glove.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)

In [0]:
# (we - they) + you = ?
glove.most_similar(positive=['you', 'we'], negative=['they'], topn=5)

In [0]:
# go : went = do : ?
glove.most_similar(positive=['do', 'went'], negative=['go'], topn=5)

In [0]:
# 근접 벡터
glove.wv.most_similar(positive="we", topn=5)

In [0]:
# 그래프를 위한 차원 제거

In [0]:
X = glove[glove.wv.vocab]
pca = PCA(n_components=2) # 2차원으로 변환 (특이값분해)
pca_results = pca.fit_transform(X)

In [0]:
plot_embeddings(words=["king", "queen", "man", "woman"], embeddings=glove, pca_results=pca_results)

# Data

## load data

In [0]:
import pandas as pd
import re
import urllib
import numpy as np

In [0]:
DATA_FILE = 'news.csv'
INPUT_FEATURE = 'title'
OUTPUT_FEATURE = 'category'

In [0]:
url = "https://raw.githubusercontent.com/madewithml/basics/master/data/news.csv"
response = urllib.request.urlopen(url)
html = response.read()
with open(DATA_FILE, 'wb') as fp:
  fp.write(html)

In [0]:
# load_data
df = pd.read_csv(DATA_FILE, header=0)
X = df[INPUT_FEATURE].values
y = df[OUTPUT_FEATURE].values
df.head(10)

# Split Data

In [0]:
import collections
# 자료를 셀 때 사용하는 객체
from sklearn.model_selection import train_test_split

In [0]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.15 # validation 데이터의 비중인듯
TEST_SIZE = 0.15
SHUFFLE = True

In [0]:
def train_val_test_split(X, y, val_size, test_size, shuffle):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, shuffle=shuffle)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, stratify=y_train, shuffle=shuffle)
  return X_train, X_val, X_test, y_train, y_val, y_test

In [0]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X = X, y=y, val_size=VAL_SIZE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [0]:
class_counts = dict(collections.Counter(y))
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"Sample point: {X_train[0]} -> {y_train[0]}")
print(f"Classes: {class_counts}")


# Tokenizer

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [0]:
FILTERS = "!\"'#$%&()*+,-./:;<=>?@[\\]^_`{|}~"
LOWER = True
CHAR_LEVEL = False # Default 가 False 인데?

In [0]:
def decode(indices, tokenizer):
  return " ".join([tokenizer.index_word[index] for index in indices])

In [0]:
# Input vectorizer
X_tokenizer = Tokenizer(filters=FILTERS, lower=LOWER, char_level=CHAR_LEVEL, oov_token='<UNK>')
# oov => out-of-vocaburay : 없는 단어는 저걸로 처리


In [0]:
X_tokenizer.fit_on_texts(X_train)
vocab_size = len(X_tokenizer.word_index) + 1
print(f"# tokens: {vocab_size}")
print(X_tokenizer.word_index)

In [0]:
original_text = X_train[0]
X_train = np.array(X_tokenizer.texts_to_sequences(X_train))
X_val = np.array(X_tokenizer.texts_to_sequences(X_val))
X_test = np.array(X_tokenizer.texts_to_sequences(X_test))
preprocessed_text = decode(X_train[0], X_tokenizer)
print(f"{original_text}\n\t=> {preprocessed_text}\n\t => {X_train[0]}")

In [0]:
X_train = np.array(X_tokenizer.texts_to_sequences(X_train))
X_train[0]

Label Encoder

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
y_tokenizer = LabelEncoder()
y_tokenizer = y_tokenizer.fit(y_train)
classes = list(y_tokenizer.classes_)
print(f"classes: {classes}")

In [0]:
# 라벨을 토큰에 매칭
y_train = y_tokenizer.transform(y_train)
y_val = y_tokenizer.transform(y_val)
y_test = y_tokenizer.transform(y_test)
print(f"y_train[0]: {y_train[0]}")


In [0]:
# class weights
counts = np.bincount(y_train) # 숫자를 세는 메소드
class_weights = {i: 1.0/count for i, count in enumerate(counts)}
print(f"class counts: {counts}, \nclass weights: {class_weights}")

## Generators

In [0]:
import math
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence

In [0]:
BATCH_SIZE = 64
FILTER_SIZES = [2,3,4]

In [0]:
# make custom data loader\
class DataGenerator(Sequence):
  def __init__(self, X, y, batch_size, max_filter_size, shuffle=False):
    self.X = X
    self.y = y
    self.batch_size = batch_size
    self.max_filter_size = max_filter_size
    self.shuffle = shuffle
    self.on_epoch_end()
  
  def __len__(self):
    # of batches...
    return math.ceil(len(self.X)/self.batch_size)

  def __str__(self):
    return(f"<DataGenerator( "\
           f"batch_size = {self.batch_size}, "\
           f"batches={len(self)}, "\
           f"shuffle={self.shuffle})>")
    
  def __getitem__(self, index):
    # Generate batch data

    # Gather indices for this batch
    batch_indices = self.epoch_indices[index * self.batch_size:(index+1)*self.batch_size]

    # 배치 데이터 생성
    X, y = self.create_batch(batch_indices=batch_indices)
    
    return X, y

  def on_epoch_end(self):
    # 매 에포크 마다 indices 생성
    self.epoch_indices = np.arange(len(self.X))
    if self.shuffle:
      np.random.shuffle(self.epoch_indices)

  def create_batch(self, batch_indices):
    # 배치를 직접적으로..

    # batch_indices 를 마스크처럼 활용
    X = self.X[batch_indices]
    y = self.y[batch_indices]

    # 패딩
    max_seq_len = max(self.max_filter_size, max([len(x) for x in X]))
    # padding="post"로 데이터의 뒷 부분에 패딩 추가 
    X = pad_sequences(X, padding="post", maxlen=max_seq_len)

    return X, y

In [0]:
# 제너레이터
training_generator = DataGenerator(X=X_train, y=y_train, batch_size=BATCH_SIZE, max_filter_size=max(FILTER_SIZES), shuffle=SHUFFLE)
validation_generator = DataGenerator(X=X_val, y=y_val, batch_size=BATCH_SIZE, max_filter_size=max(FILTER_SIZES), shuffle=False)
testing_generator = DataGenerator(X=X_test, y=y_test, batch_size=BATCH_SIZE, max_filter_size=max(FILTER_SIZES), shuffle=False)


In [0]:
print(f"training_generator:{training_generator}")
print(f"validation_generator:{validation_generator}")
print(f"testing_generator:{testing_generator}")

In [0]:
# 사용해보기
print(f"num test batches: {len(testing_generator)}")
for i, batch in enumerate(testing_generator):
  if i >= 10: break
  print(f"batch{i} | x: {batch[0].shape}, y: {batch[1].shape}")

# Embeddings

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras import Input

In [0]:
x = Input(shape=(10)) # (batch_size, max_seq_len)

In [0]:
x


In [0]:
# Embeddings
embedding = Embedding(input_dim=10, # vocab size
                      output_dim=100 # embedding size
                      )

In [0]:
embedding(x).shape

# Modeling

# Model

In [0]:
from tensorflow.keras.layers import Concatenate, Conv1D, Dense, Dropout, Embedding, GlobalMaxPool1D, Input
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [0]:
class CNN(Model):
  def __init__(self, vocab_size, embedding_dim, filter_sizes, num_filters, hidden_dim, dropout_p, num_classes, freeze_embeddings=False):
    super(CNN, self).__init__(name="cnn")

    # Embeddings
    self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, trainable=not freeze_embeddings)

    # Convolutional filters
    self.convs = []
    self.pools = []
    for filter_size in filter_sizes:
      conv = Conv1D(filters=num_filters, kernal_size = filter_size, padding='same', activation="relu")
      pool = GlobalMaxPool1D(data_format='channels_last')
      self.convs.append(conv)
      self.pools.append(pool)

    self.cancat = Concatenate(axis=1)

    # FC layers
    self.fc1 = Dense(units=hidden_dim, activation='relu')
    self.dropout = Dropout(rate=dropout_p)
    self.fc2 = Dense(units=num_classes, activation='softmax')

def call(self, x_in, training=False):
  # Embedding
  x_emb = self.embedding(x_in)

  # Convoultions
  convs = []
  for i in range(len(self.convs)):
    z = self.convs[i](x_emb)
    z = self.pools[i](z)
    convs.append(z)
  
  # Concatenate
  z_cat = self.concat(convs)

  # Fc
  z = self.fc1(z_cat)
  if training:
    # 드롭아웃은 훈련때만 적용
    z = self.dropout(z, training = training)
  y_pred = self.fc2(z)

  return y_pred

def summary(self, input_shape):
  x_in = Input(shape=input_shape, name='X')
  summary = Model(inputs=x_in, outputs=self.call(x_in), name=self.name)
  return plot_model(summary, show_shapes=True)

## gloVe embeddings

In [0]:
def load_glove_embeddings(embeddings_file):
  embeddings = {}
  with open(embeddings_file, "r") as fp:
    for index, line in enumerate(fp):
      values = line.split()
      word = values[0]
      embedding = np.asarray(values[1:], dtype='float32')
      embeddings[word] = embedding
  return embeddings

In [0]:
def make_embeddings_matrix(embeddings, word_index, embedding_dim):
  embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
  for word, i in word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

  return embedding_matrix

In [0]:
embedding_file = 'glove.6B.{0}d.txt'.format(EMBEDDING_DIM)
glove_embeddings = load_glove_embeddings(embeddings_file=embeddings_file)
embedding_matrix = make_embeddings_matrix(embeddings=glove_embeddings,
                                          word_index=X_tokenizer.word_index,
                                          embedding_dim=EMBEDDING_DIM)
print(f"<Embeddings(words={embedding_matrix.shape[0]}, dim={embedding_matrix.shape[1]})>")